# 라이브러리 설치

In [ ]:
pip install fastapi uvicorn ultralytics opencv-python-headless numpy python-multipart streamlit

# FastAPI application 만들기

In [3]:
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np
from ultralytics import YOLO

# Initialize FastAPI app
app = FastAPI()

# Load YOLOv8 model
model = YOLO("C:/Users/gh159/runs/detect/yolov8_custom8/weights/best.pt")

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Read uploaded image
    image_bytes = await file.read()
    image = np.frombuffer(image_bytes, dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    # Perform object detection
    results = model.predict(image)

    # Process detection results
    detections = []
    for result in results[0].boxes:
        x1, y1, x2, y2 = result.xyxy.tolist()
        confidence = result.conf.tolist()
        class_id = result.cls.tolist()
        detections.append({
            "x1": x1,
            "y1": y1,
            "x2": x2,
            "y2": y2,
            "confidence": confidence,
            "class_id": class_id,
        })

    return {"detections": detections}


In [2]:
import requests
import base64

# 테스트 이미지 로드 및 base64 인코딩
with open("testnormal.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

# API 요청 보내기
url = "http://127.0.0.1:8000/predict/"
data = {"image_base64": encoded_string}
response = requests.post(url, json=data)

# 결과 출력
if response.status_code == 200:
    result = response.json()
    print("Predictions:", result["predictions"])
else:
    print("Error:", response.text)


Predictions: [{'label': 'Defective', 'confidence': 96.24550342559814, 'bbox': [498, 808, 3337, 2162]}]


In [ ]:
import streamlit as st
import requests
import base64
from PIL import Image
from io import BytesIO

# FastAPI 서버 URL 설정
FASTAPI_URL = "http://127.0.0.1:8000/predict/"

# Streamlit 앱 시작
st.title("YOLOv8 객체 탐지")
st.write("이미지를 업로드하면 객체 탐지 결과를 확인할 수 있습니다.")

# 이미지 업로드 위젯
uploaded_file = st.file_uploader("이미지를 업로드하세요", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # 업로드된 파일을 FastAPI로 전송
    with st.spinner("FastAPI로 이미지 전송 중..."):
        files = {"file": uploaded_file.getvalue()}
        response = requests.post(FASTAPI_URL, files=files)

    if response.status_code == 200:
        # FastAPI에서 반환된 base64 이미지를 디코딩하여 표시
        result = response.json()
        annotated_image_base64 = result["annotated_image"]
        
        # base64 이미지를 PIL 이미지로 변환하여 Streamlit에 표시
        annotated_image_bytes = base64.b64decode(annotated_image_base64)
        annotated_image = Image.open(BytesIO(annotated_image_bytes))
        
        st.image(annotated_image, caption="탐지 결과", use_column_width=True)
    else:
        st.error(f"FastAPI 오류: {response.status_code}, {response.text}")
